In [ ]:
class DerivedVariable:
    
    '''
    파생변수를 만드는 Class 입니다.
    아래의 방법으로 파생변수를 생성하였습니다.
    
    date         : dtype을 datetime 으로 변환
    score        : (rank * 1) + (kill * 2) + (assist * 2)
    tier         : score column을 바탕으로 구함. squad의 +0.6은 위치를 맞추기 위한 수평이동
    KDA          : kill, assist, death를 이용해 파생변수 생성 
                   death는 match 합계에서 team_placement가 1등인 횟수를 제외하여 계산
    num_of_match : player별 총 게임 횟수를 계산
    
    '''
    def date(self, df):
        df['date'] = pd.to_datetime(df['date'])
    
    def score(self, df):
        # player 별 score 
        if df.loc[0, 'party_size'] == 1 :
            df['each_game_score'] = (100 - df['team_placement'])*1 + df['player_kills']*2 + df['player_assists']*2
        elif df.loc[0, 'party_size'] == 2 :
            df['each_game_score'] = (50 - df['team_placement'])*1 + df['player_kills']*2 + df['player_assists']*2
        elif df.loc[0, 'party_size'] == 4 :
            df['each_game_score'] = (36 - df['team_placement'])*1 + df['player_kills']*2 + df['player_assists']*2
        
        # score의 sum 
        score = df.groupby(['player_name', 'match_id'])['each_game_score'].first().to_frame()
        total_score = score.groupby('player_name')['each_game_score'].sum().to_frame()
        total_score.columns = ['total_score']
        df['total_score'] = total_score.loc[df['player_name'], 'total_score'].values
        
        return df
    
    def tier(self, df):
        # score의 합의 log scailing과 squad의 그래프 수평이동
        if df.loc[0, 'party_size'] == 4 :
            df['total_score_log'] = np.log(df['total_score']) + 0.6
        else:
            df['total_score_log'] = np.log(df['total_score']) 
        
        # tier 별 분할
        scores = [(df['tear_score'].min() - 1), 6, 6.75, 7.5, 8.25, 9, df['tear_score'].max()]
        tears = [1, 2, 3, 4, 5, 6]
        df['tear'] = pd.cut(df['tear_score'], bins = scores, labels = tears)
        
        return df
        # 이렇게 하면 1~6이 숫자가 아니라 카테고리로 들어갑니다. 체크 요망!
    
    def kda_num_of_match(self, df):
        # 필요한 column 만 추출
        anal_col = df[['player_name', 'match_id','team_placement', 'player_kills', 'player_assists']]

        # num_of_match
        num_of_match = anal_col.groupby('player_name')['match_id'].nunique().reset_index()
        num_of_match.columns = ['player_name', 'num_of_match']

        # num_of_1st
        num_of_1st = anal_col[anal_col['team_placement'] == 1].groupby('player_name')['match_id'].nunique().reset_index()
        num_of_1st.columns = ['player_name', 'num_of_1st']

        # death 
        temp = num_of_match.merge(num_of_1st, how = 'left', on = 'player_name')
        temp = temp.fillna(0)
        temp['deaths_sum'] = temp['num_of_match'] - temp['num_of_1st']
        death = temp[['player_name', 'deaths_sum']]

        # kill & assist
        KA_count = anal_col.groupby(['player_name', 'match_id'])[['player_kills', 'player_assists']].first().reset_index()
        KA_sum = KA_count.groupby('player_name')[['player_kills', 'player_assists']].sum().reset_index()
        KA_sum.columns = ['player_name', 'kills_sum', 'assists_sum']

        # KDA
        KDA = KA_sum.merge(death, how = 'left', on = 'player_name')
        KDA['kda'] = (KDA['kills_sum'] + KDA['assists_sum']) / KDA['deaths_sum']
        KDA_1 = KDA[['player_name', 'kda']]


        ## 여기 한번에 표현 할 수 있는가
        df = df.merge(KDA_1, how='left', on='player_name')
        df = df.merge(num_of_match, how='left', on='player_name' )
        df = df.loc[df['num_of_match'] >= 10]

        return df
